In [ ]:
"""satellites_unet.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/142XiiXOko5VDq93Y5DF3GKaOo6MoUclm
"""

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torch.optim as optim
import torch.nn.functional as F



from sklearn.model_selection import train_test_split

import skimage.io
import pandas as pd
from pandas import array

from tensorflow.keras.utils import normalize
import os
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
import glob

from tensorflow.keras.metrics import MeanIoU

from skimage.color import rgb2gray


In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate

### Défintion des blocs ###

# Bloc de convolution (2 conv and batch layers) #
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network.
    x = Activation("relu")(x)
    x = Dropout(0.25)(x)
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    return x


# Bloc de l'encoder #
def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p


# Bloc du décoder #
def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

# Structure/Assemblage des blocs --> de U-Net #
def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    if n_classes == 1:
      activation = 'sigmoid'
    else:
      activation = 'softmax'

    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model


# my_unet = build_unet(input_shape=(256,256,3), n_classes=3)
# my_unet.summary()




In [ ]:
import random

SIZE = 256
num_images = 1200

# Data Paths
ROOT_IMAGE_DIR = '/content/drive/MyDrive/Forest Segmented/images/'
ROOT_MASK_DIR = '/content/drive/MyDrive/Forest Segmented/masks/'
METADATA_CSV_PATH = '/content/drive/MyDrive/Forest Segmented/meta_data.csv'

# Read the images
arrayed_images = []
arrayed_masks = []


image_names = glob.glob("/content/drive/MyDrive/Forest Segmented/images/*.jpg")
image_names_subset = image_names[0:num_images]
print(image_names[0])


for i,img in enumerate(image_names_subset):
    img = Image.open(img)
    array_img = np.array(img)
    arrayed_images.append(array_img)

mask_names = glob.glob("/content/drive/MyDrive/Forest Segmented/masks/*.jpg")
mask_names_subset = mask_names[0:num_images]


for i,img in enumerate(mask_names_subset):
    img = Image.open(img)
    array_img = np.array(img)
    arrayed_masks.append(array_img)

import csv
# Chargement des correspondances depuis le fichier CSV
correspondances = {}
with open('/content/drive/MyDrive/Forest Segmented/meta_data.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for i,row in enumerate(reader):
        correspondances[row[0]] = row[1]


print(correspondances)

# Création de deux nouvelles listes triées en fonction des correspondances
sorted_images = []
sorted_masks = []
for i, image in enumerate(image_names_subset):
    image = image.split("/")[-1]
    sorted_images.append(image)
    sorted_masks.append(correspondances[image])
    # if i == 99 :
    #   break;

X_predictions = []
Y_predictions = []
arrayed_sorted_images = []
for i,img in enumerate(sorted_images):
    img = ROOT_IMAGE_DIR + img
    img = Image.open(img)
    array_img = np.array(img)
    #array_img = cv2.cvtColor(array_img, cv2.COLOR_BGR2GRAY)
    array_img = rgb2gray(img)
    if i < 1000 :
      arrayed_sorted_images.append(array_img)
    else :
      X_predictions.append(array_img)

arrayed_sorted_masks = []
for i,img in enumerate(sorted_masks):
    img = ROOT_MASK_DIR + img
    img = Image.open(img)
    array_img = np.array(img)
    #array_img = cv2.cvtColor(array_img, cv2.COLOR_BGR2GRAY)
    array_img = rgb2gray(img)
    if i < 1000 :
      arrayed_sorted_masks.append(array_img)
    else :
      Y_predictions.append(array_img)


dataset = np.expand_dims(arrayed_sorted_images, axis = 3)
maskset = np.expand_dims(arrayed_sorted_masks, axis = 3)
predictionset_x = np.expand_dims(X_predictions, axis = 3)
predictionset_y = np.expand_dims(Y_predictions, axis = 3)

print("Image data shape is: ", dataset.shape)
print("mask data shape is: ", maskset.shape)
print("prediction Image data shape is: ", predictionset_x.shape)
print("prediction mask data shape is: ", predictionset_y.shape)
print("Max pixel value in image is: ", dataset.max())


X_train, X_test, y_train, y_test = train_test_split(dataset, maskset, test_size = 0.2, random_state = 42)


# --- MODELE UNET, TRAINING DAY --- #

mean_iou = MeanIoU(num_classes=2, name="MeanIoU")

my_unet = build_unet(input_shape=(256,256,1), n_classes=1)
my_unet.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy',metrics=[mean_iou,'accuracy'])
print(my_unet.summary())


history = my_unet.fit(X_train, y_train,
                    batch_size = 1,
                    verbose= 1,
                    epochs= 20,
                    validation_data=(X_test, y_test),
                    shuffle=False)

#Save the model for future use
print('training done, saving the model')
#my_unet.save('/content/drive/MyDrive/Forest Segmented/sauvegarde_1200_binary_crossentropy_train_withDropout_butWithoutShuffle_20epochs.hdf5')

#Load previously saved model
from keras.models import load_model
#my_unet = load_model("/content/drive/MyDrive/Forest Segmented/sauvegarde_1200_binary_crossentropy_train_withDropout_butWithoutShuffle_50epochs.hdf5", compile=False)

# plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

meaniou = history.history['MeanIoU']
val_meaniou = history.history['val_MeanIoU']
plt.plot(epochs, meaniou, 'y', label='Training meaniou')
plt.plot(epochs, val_meaniou, 'r', label='Validation meaniou')
plt.title('Training and validation meaniou')
plt.xlabel('Epochs')
plt.ylabel('meaniou')
plt.legend()
plt.show()

#IOU
y_pred=my_unet.predict(predictionset_x)
y_pred_thresholded = y_pred > 0.5 # assigner une valeur binaire à chaque pixel d'une image en fonction d'une valeur de seuil, ici 0.5 => x<0.5 -> 0 et x>0.5 ->1


n_classes = 2
IOU_keras = MeanIoU(num_classes=n_classes)
IOU_keras.update_state(y_pred_thresholded, y_test)
#print("Mean IoU =", IOU_keras.result().numpy())


threshold = 0.5
test_img_number = random.randint(0, len(predictionset_x)-1)

# test_img = X_test[test_img_number]
# ground_truth=y_test[test_img_number]
test_img = predictionset_x[25]
ground_truth=predictionset_y[25]

# test_img = predictionset_x[test_img_number]
# ground_truth= predictionset_y[test_img_number]
test_img_input=np.expand_dims(test_img, 0)
#print(test_img_input.shape)
prediction = (my_unet.predict(test_img_input)[0,:,:,0] > 0.5).astype(np.uint8)
print("Random prediction image has shape :" ,prediction.shape, "and has Mean IoU =", IOU_keras.result().numpy())

plt.figure(figsize=(16, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img)
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth)
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(prediction, cmap='gray')

plt.show()